In [1]:
# All  import statements needed for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/models/model_split_softmax_cnn_model.hdf5')

In [5]:
data = pd.read_csv('/content/drive/MyDrive/balancedhalf_data.csv')
data.drop(data.columns[0] , inplace=True , axis=1)
unknown = ['SSH-Patator','DoS slowloris','DoS Slowhttptest','Bot','Infiltration','Heartbleed']
att = data.loc[(data['Label'].isin(unknown))]
data.drop(att.index,axis=0 , inplace=True, errors='ignore')
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns
# using standard scaler for normalizing
std_scaler = MinMaxScaler()
def normalization(df,att,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    x = np.array(att[i])
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    #To use the same scaler which was used in preprocessing the train data 
    att[i] = std_scaler.transform(x.reshape(len(x),1))
  return df,att
# calling the normalization() function
data , att = normalization(data.copy(),att.copy(),numeric_col)
att.Label = 'unknown'
X = att.drop('Label' , axis=1)
X = X.to_numpy().reshape(-1, 83,1)

y = att.Label
X_train = data.drop('Label' , axis=1)
X_train = X_train.to_numpy().reshape(-1, 83,1)

y_train = data.Label

In [6]:
predicted = model.predict(X)
Y_train_predicted = model.predict(X_train)

34117/34117 [==============================] - 77s 2ms/step


In [7]:
def new_probailities_set(prob):
    import numpy as np
    prob = np.array(prob)
    newp = np.prod(1-prob)
    prob = prob/(1+newp) ## for existing probabilty
    newp = newp/(1+newp)
    prob = np.append(prob,newp)
    return prob

In [8]:
type(predicted) , type(Y_train_predicted)

(numpy.ndarray, numpy.ndarray)

In [9]:
predicted = np.apply_along_axis(new_probailities_set,1,predicted)

In [10]:
Y_train_predicted = np.apply_along_axis(new_probailities_set,1,Y_train_predicted)

In [11]:
my_dict = {
    0 : 'BENIGN',
    1 : 'DDoS',
    2 : 'DoS Hulk',
    3 : 'DoS GoldenEye',
    4 : 'PortScan',
    5 : 'FTP-Patator',
    6:  'Unknown'
}
max_probability = np.amax(predicted, axis = 1)
unknown_index = np.argmax(predicted, axis = 1)
unknown_index_label = np.vectorize(my_dict.get)(unknown_index)
max_probability_train = np.amax(Y_train_predicted, axis = 1)
y_prediction_index = np.argmax(Y_train_predicted, axis = 1)
y_prediction_test_label = np.vectorize(my_dict.get)(y_prediction_index)
RIGHT_BOOLEAN = y_train == y_prediction_test_label

In [12]:
TOTAL = data.shape[0]+att.shape[0]
## calculate no of right predictions
truepredicted = 0
# 1.0 TRAIN SET:
truepredicted += RIGHT_BOOLEAN.sum() ##simply check no of trues in train set...
# 2.0 unknown set
##here we have to check how many indices are unknown
UNKNOWN_RIGHT_BOOLEAN = unknown_index_label == 'Unknown'
truepredicted += UNKNOWN_RIGHT_BOOLEAN.sum()

## ACCURACY PREDICTION:
Accuracy = truepredicted/TOTAL
print('Accuracy is ',Accuracy)

Accuracy is  0.6149170246243695


In [13]:
UNKNOWN_RIGHT_BOOLEAN.sum() ,  att.shape[0]

(87, 19205)

In [14]:
RIGHT_BOOLEAN.sum() , data.shape[0]

(683044, 1091727)